In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import requests
import json
import folium
from pandas.io.json import json_normalize

In [15]:
# 따릉이 서버에 post 방식으로 요청해서 서버가 응답하는 데이터를 받는다.
targetSite = 'https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do'
request = requests.post(targetSite, data={
    'stationGrpSeq': 'ALL'
})
# print(request) # <Response [200]>
# print(type(request.text)) # <class 'str'>
# print(request.text)

In [16]:
# 따릉이 서버가 응답하는 json 형태의 문자열을 파이썬에서 사용하기 위해서 딕셔너리 타입으로 변환한다.
# bike_json = json.loads(request.text) # json 모듈의 loads() 메소드 사용
bike_json = request.json() # requests 모듈의 json() 메소드 사용
# print(type(bike_json))
# print(bike_json.keys())
# print(bike_json)

In [14]:
# bike_json['realtimeList']
bike_json.get('realtimeList')

[{'stationId': 'ST-4',
  'stationImgFileName': '',
  'stationName': '102. 망원역 1번출구 앞',
  'parkingBikeTotCnt': '0',
  'parkingQRBikeCnt': '8',
  'parkingELECBikeCnt': '12',
  'stationSeCd': 'RAK_002',
  'stationLongitude': '126.91062927',
  'stationLatitude': '37.55564880',
  'rackTotCnt': '15',
  'mode': None},
 {'stationId': 'ST-5',
  'stationImgFileName': '',
  'stationName': '103. 망원역 2번출구 앞',
  'parkingBikeTotCnt': '0',
  'parkingQRBikeCnt': '3',
  'parkingELECBikeCnt': '11',
  'stationSeCd': 'RAK_002',
  'stationLongitude': '126.91083527',
  'stationLatitude': '37.55495071',
  'rackTotCnt': '14',
  'mode': None},
 {'stationId': 'ST-6',
  'stationImgFileName': '',
  'stationName': '104. 합정역 1번출구 앞',
  'parkingBikeTotCnt': '0',
  'parkingQRBikeCnt': '4',
  'parkingELECBikeCnt': '0',
  'stationSeCd': 'RAK_002',
  'stationLongitude': '126.91508484',
  'stationLatitude': '37.55073929',
  'rackTotCnt': '13',
  'mode': None},
 {'stationId': 'ST-7',
  'stationImgFileName': '',
  'stationN

In [20]:
# pandas.io.json 모듈의 json_normalize() 메소드로 json 타입의 데이터가 변환된 딕셔너리를 데이터프레임으로 변환한다.
# json_normalize(딕셔너리, 데이터프레임으로 변환할 데이터가 할당된 딕셔너리의 key)
bike_df = json_normalize(bike_json, 'realtimeList')
bike_df

,stationId,stationImgFileName,stationName,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt,stationSeCd,stationLongitude,stationLatitude,rackTotCnt,mode
0,ST-4,,102. 망원역 1번출구 앞,0,7,12,RAK_002,126.91062927,37.55564880,15,None
1,ST-5,,103. 망원역 2번출구 앞,0,4,11,RAK_002,126.91083527,37.55495071,14,None
2,ST-6,,104. 합정역 1번출구 앞,0,4,0,RAK_002,126.91508484,37.55073929,13,None
3,ST-7,,105. 합정역 5번출구 앞,0,0,0,RAK_002,126.91482544,37.55000687,5,None
4,ST-8,,106. 합정역 7번출구 앞,0,7,0,RAK_002,126.91282654,37.54864502,12,None
...,...,...,...,...,...,...,...,...,...,...,...
2723,ST-3297,,6058. 서울도시건축전시관 옆,0,9,1,RAK_002,126.97684479,37.56653976,10,None
2724,ST-3297,,6058. 서울도시건축전시관 옆,0,9,1,RAK_002,126.97684479,37.56653976,10,None
2725,ST-3276,,6171. 월드빌딩 앞,0,22,0,RAK_002,126.83743286,37.54098129,12,None
2726,ST-3281,,6172. 가양5단지아파트,0,3,0,RAK_002,126.85464478,37.56447983,10,None


In [18]:
bike_df.columns

Index(['stationId', 'stationImgFileName', 'stationName', 'parkingBikeTotCnt',
       'parkingQRBikeCnt', 'parkingELECBikeCnt', 'stationSeCd',
       'stationLongitude', 'stationLatitude', 'rackTotCnt', 'mode'],
      dtype='object')

In [21]:
# stationId: 대여소 id
# stationName: 대여소 이름
# parkingBikeTotCnt: 주차된 따릉이 LCD형 대수 => 사용하지 않음
# parkingQRBikeCnt: 주차된 따릉이 QR형 대수 => 일반 따릉이
# parkingELECBikeCnt: 주차된 새싹 따릉이 대수
# stationLongitude: 대여소 경도
# stationLatitude: 대여소 위도
# rackTotCnt: 주차 가능한 자전거 대수

In [23]:
bike_df_map = bike_df[['stationId', 'stationName', 'parkingQRBikeCnt', 'parkingELECBikeCnt', 
                       'stationLongitude', 'stationLatitude']]
bike_df_map

,stationId,stationName,parkingQRBikeCnt,parkingELECBikeCnt,stationLongitude,stationLatitude
0,ST-4,102. 망원역 1번출구 앞,7,12,126.91062927,37.55564880
1,ST-5,103. 망원역 2번출구 앞,4,11,126.91083527,37.55495071
2,ST-6,104. 합정역 1번출구 앞,4,0,126.91508484,37.55073929
3,ST-7,105. 합정역 5번출구 앞,0,0,126.91482544,37.55000687
4,ST-8,106. 합정역 7번출구 앞,7,0,126.91282654,37.54864502
...,...,...,...,...,...,...
2723,ST-3297,6058. 서울도시건축전시관 옆,9,1,126.97684479,37.56653976
2724,ST-3297,6058. 서울도시건축전시관 옆,9,1,126.97684479,37.56653976
2725,ST-3276,6171. 월드빌딩 앞,22,0,126.83743286,37.54098129
2726,ST-3281,6172. 가양5단지아파트,3,0,126.85464478,37.56447983
